## 1. Mount Google Drive and Install Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install SimpleITK openpyxl pyradiomics

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 12.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
# prompt: instal optuna

!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 10.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold, cross_val_score, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Cargar características GLCM y etiquetas
# Ajusta los nombres de clave según tu .npz
data = np.load('/content/slice_glcm1d.npz',allow_pickle=True)

In [ ]:
import pandas as pd
data["slice_features"].shape

(7414, 24)

slice_features: ndarray of dimensions (Nslices,24) containing the 24 GLCM features for each slice of each nodule

slice_meta: ndarray containing metada of each slice, filename, patient_id, nodule_id, diagnosis = {'Benign' | 'Malignant' | 'NoNod' (if slice mask pixels < 200)}:

 ['LIDC-IDRI-0001_GT1_1', 1, 1, 'Malignant']

features_ranking_idx: index of GLCM features according p-value of a t-test (sorted from most to least important) of each of the 24 GLCM features. Those with p-value below the significance level (0.01,0.05) are not relevant for the diagnosis ('Benign' vs 'Malignant')

In [ ]:
df = pd.DataFrame(data['slice_features'])
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,3912.728918,1.096981e+07,38704.669443,2122.761852,236.130245,0.799733,8.958410,1.280547,153.858318,0.668218,...,0.874570,0.000520,58.659495,0.149956,3.108626,0.818609,0.276677,117.318989,2.763986,589.723024
1,7511.568847,3.138924e+07,-19143.340098,4128.827862,188.601517,0.912418,8.283005,1.331034,117.985636,0.640403,...,0.953910,0.000812,80.784854,0.085426,3.834779,0.918409,0.139321,161.569708,3.407020,1079.357345
2,8858.349761,3.493607e+07,-197109.877954,3355.497691,223.661489,0.874257,8.822815,1.381786,143.925152,0.639497,...,0.919476,0.000775,89.861605,0.186931,3.461811,0.892283,0.405126,179.723210,3.044884,894.789795
3,9830.155057,1.601792e+07,-118166.648120,1794.384095,208.654995,0.789844,7.819523,1.297230,146.325976,0.689305,...,0.826478,0.000654,97.126247,0.315186,2.823842,0.817015,0.548032,194.252495,2.435366,500.759772
4,9932.448352,2.210879e+07,-151369.529909,1948.802779,209.900402,0.803567,7.611905,1.277180,150.267237,0.705191,...,0.824300,0.000604,97.455314,0.344902,2.748613,0.826901,0.577190,194.910628,2.368467,539.675795


In [ ]:
# Cargar datos
data = np.load("slice_glcm1d.npz", allow_pickle=True)
X = data["slice_features"]
meta = data["slice_meta"]
ranking_idx = data["features_rankin_idx"]

# ------
print(data.files)
print(meta[:5])
print(X.shape)


['slice_features', 'slice_meta', 'features_rankin_idx']
[['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
 ['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
 ['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
 ['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']
 ['LIDC-IDRI-0001_GT1_1' 1 1 'Malignant']]
(7414, 24)


### Baseline model (svm with stratifiedkfold)

##### data spliting and validation skfold and s grouped kfold

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold

# === 1. Load all features & metadata ===
data = np.load("slice_glcm1d.npz", allow_pickle=True)
X_full = data["slice_features"]        # shape (n_samples, n_features)
meta   = data["slice_meta"]            # shape (n_samples, 4): [ID, ?, nodule_id, label]

# === 2. Filter only Benign vs Malignant ===
labels_all = meta[:, 3]
mask       = (labels_all == "Benign") | (labels_all == "Malignant")
X          = X_full[mask]
y          = np.where(labels_all[mask] == "Malignant", 1, 0)

# === 3. Extract group IDs (nodule) ===
groups_all = meta[:, 2].astype(int)
groups     = groups_all[mask]

# === 4. Build pipeline ===
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("svc",    SVC())
])

# === 5. Train & evaluate with StratifiedKFold (by slice class) ===
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf_accuracies = []
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    skf_accuracies.append(acc)
    print(f"SKFold Fold {fold}: accuracy = {acc:.3f}")
print(f"SKFold Mean accuracy: {np.mean(skf_accuracies):.3f} ± {np.std(skf_accuracies):.3f}\n")

# === 6. Train & evaluate with StratifiedGroupKFold (by nodule) ===
gkf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
gkf_accuracies = []
for fold, (train_idx, test_idx) in enumerate(gkf.split(X, y, groups), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    gkf_accuracies.append(acc)
    print(f"GKFold Fold {fold}: accuracy = {acc:.3f}")
print(f"GKFold Mean accuracy: {np.mean(gkf_accuracies):.3f} ± {np.std(gkf_accuracies):.3f}")


SKFold Fold 1: accuracy = 0.908
SKFold Fold 2: accuracy = 0.901
SKFold Fold 3: accuracy = 0.910
SKFold Fold 4: accuracy = 0.908
SKFold Fold 5: accuracy = 0.916
SKFold Mean accuracy: 0.909 ± 0.005

GKFold Fold 1: accuracy = 0.943
GKFold Fold 2: accuracy = 1.000
GKFold Fold 3: accuracy = 1.000
GKFold Fold 4: accuracy = 0.896
GKFold Fold 5: accuracy = 0.838
GKFold Mean accuracy: 0.935 ± 0.062


##stratified kfold group

### Hyperparameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from scipy.stats import f_oneway

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__gamma': [0.001, 0.01, 0.1, 1],
    'svc__kernel': ['rbf']
}

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

grid_search = GridSearchCV(pipeline, param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
grid_search.fit(X, y)

print(" Best parameters (Grid Search):", grid_search.best_params_)
print(" Best accuracy:", grid_search.best_score_)
# Extraer resultados del grid search
results = pd.DataFrame(grid_search.cv_results_)

# Mostrar tabla con medias y std por combinación
print("\n Resultados por combinación:")
summary = results[['params', 'mean_test_score', 'std_test_score']]
print(summary)

# ANOVA: agrupamos por combinaciones y extraemos las 5 accuracies de cada una
groups = []

for mean, std, idx in zip(results['mean_test_score'], results['std_test_score'], results.index):
    scores = results.loc[idx, 'split0_test_score':'split4_test_score'].values
    groups.append(scores)

# Aplicar ANOVA
f_stat, p_value = f_oneway(*groups)
print("\n ANOVA:")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4e}")

if p_value < 0.05:
    print("→ Hay diferencias estadísticamente significativas entre las combinaciones de parámetros.")
else:
    print("→ No hay diferencias estadísticamente significativas entre las combinaciones de parámetros.")


 Best parameters (Grid Search): {'svc__C': 10, 'svc__gamma': 0.1, 'svc__kernel': 'rbf'}
 Best accuracy: 0.9232087803674929

 Resultados por combinación:
                                               params  mean_test_score  \
0   {'svc__C': 0.1, 'svc__gamma': 0.001, 'svc__ker...         0.897188   
1   {'svc__C': 0.1, 'svc__gamma': 0.01, 'svc__kern...         0.897188   
2   {'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kerne...         0.900545   
3   {'svc__C': 0.1, 'svc__gamma': 1, 'svc__kernel'...         0.897188   
4   {'svc__C': 1, 'svc__gamma': 0.001, 'svc__kerne...         0.897188   
5   {'svc__C': 1, 'svc__gamma': 0.01, 'svc__kernel...         0.908521   
6   {'svc__C': 1, 'svc__gamma': 0.1, 'svc__kernel'...         0.915236   
7   {'svc__C': 1, 'svc__gamma': 1, 'svc__kernel': ...         0.914815   
8   {'svc__C': 10, 'svc__gamma': 0.001, 'svc__kern...         0.906001   
9   {'svc__C': 10, 'svc__gamma': 0.01, 'svc__kerne...         0.909781   
10  {'svc__C': 10, 'svc__gamma': 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'svc__C': uniform(0.01, 100),
    'svc__gamma': uniform(0.001, 1),
    'svc__kernel': ['rbf']
}

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_dist,
    n_iter=30,
    cv=skf,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)

random_search.fit(X, y)

print(" Best parameters (Random Search):", random_search.best_params_)
print(" Best accuracy:", random_search.best_score_)

# Extraer resultados del grid search
results = pd.DataFrame(random_search.cv_results_)

# Mostrar tabla con medias y std por combinación
print("\n Resultados por combinación:")
summary = results[['params', 'mean_test_score', 'std_test_score']]
print(summary)

# ANOVA: agrupamos por combinaciones y extraemos las 5 accuracies de cada una
groups = []

for mean, std, idx in zip(results['mean_test_score'], results['std_test_score'], results.index):
    scores = results.loc[idx, 'split0_test_score':'split4_test_score'].values
    groups.append(scores)

# Aplicar ANOVA
f_stat, p_value = f_oneway(*groups)
print("\n ANOVA:")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4e}")

if p_value < 0.05:
    print("→ Hay diferencias estadísticamente significativas entre las combinaciones de parámetros.")
else:
    print("→ No hay diferencias estadísticamente significativas entre las combinaciones de parámetros.")


 Best parameters (Random Search): {'svc__C': np.float64(4.532728891053806), 'svc__gamma': np.float64(0.32633033076326434), 'svc__kernel': 'rbf'}
 Best accuracy: 0.9248868100699401

 Resultados por combinación:
                                               params  mean_test_score  \
0   {'svc__C': 37.464011884736244, 'svc__gamma': 0...         0.913973   
1   {'svc__C': 73.20939418114051, 'svc__gamma': 0....         0.910619   
2   {'svc__C': 15.611864044243651, 'svc__gamma': 0...         0.921111   
3   {'svc__C': 5.818361216819946, 'svc__gamma': 0....         0.917333   
4   {'svc__C': 60.121501174320876, 'svc__gamma': 0...         0.913973   
5   {'svc__C': 2.0684494295802445, 'svc__gamma': 0...         0.921107   
6   {'svc__C': 83.25426408004218, 'svc__gamma': 0....         0.904744   
7   {'svc__C': 18.192496720710064, 'svc__gamma': 0...         0.918594   
8   {'svc__C': 30.434224295953772, 'svc__gamma': 0...         0.908943   
9   {'svc__C': 43.20450186421157, 'svc__gamma': 0.

In [ ]:
import optuna
from sklearn.model_selection import cross_val_score

# Optuna Objective
def objective(trial):
    C = trial.suggest_loguniform('C', 0.01, 100)
    gamma = trial.suggest_loguniform('gamma', 0.001, 1)

    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(C=C, gamma=gamma, kernel='rbf'))
    ])

    scores = cross_val_score(pipeline, X, y, cv=skf, scoring='accuracy', n_jobs=-1)
    trial.set_user_attr("scores", scores)  # guardar los scores para ANOVA después
    return scores.mean()

# Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Resultados
print(" Best parameters (Optuna):", study.best_params)
print(" Best mean accuracy:", study.best_value)

# Obtener scores por trial
trial_scores = []
for trial in study.trials:
    if trial.user_attrs.get("scores") is not None:
        trial_scores.append(trial.user_attrs["scores"])

# Calcular accuracy media y std del mejor trial
best_trial = study.best_trial
best_scores = best_trial.user_attrs["scores"]
print("\n Resultados del mejor trial:")
print("Accuracy media:", np.mean(best_scores))
print("Desviación estándar:", np.std(best_scores))

# ANOVA
f_stat, p_value = f_oneway(*trial_scores)
print("\n ANOVA entre los trials:")
print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4e}")

if p_value < 0.05:
    print("→ Hay diferencias estadísticamente significativas entre las combinaciones de hiperparámetros evaluadas.")
else:
    print("→ No hay diferencias estadísticamente significativas entre las combinaciones de hiperparámetros.")

[I 2025-05-19 14:46:14,506] A new study created in memory with name: no-name-e8eb8db0-a3ea-41cb-9962-e3e68fb95902
<ipython-input-41-292a6f01c3ed>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 0.01, 100)
<ipython-input-41-292a6f01c3ed>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 0.001, 1)
[I 2025-05-19 14:46:15,655] Trial 0 finished with value: 0.907260010922608 and parameters: {'C': 24.637193102745236, 'gamma': 0.001117557877204178}. Best is trial 0 with value: 0.907260010922608.
<ipython-input-41-292a6f01c3ed>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feat

🔧 Best parameters (Optuna): {'C': 3.7682229501522895, 'gamma': 0.43349608037029486}
✅ Best mean accuracy: 0.9248912143473742

📊 Resultados del mejor trial:
Accuracy media: 0.9248912143473742
Desviación estándar: 0.011086053124067415

📈 ANOVA entre los trials:
F-statistic: 8.5775
P-value: 6.1382e-18
→ Hay diferencias estadísticamente significativas entre las combinaciones de hiperparámetros evaluadas.
